In [ ]:
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import os

In [ ]:
import sliderule
from sliderule import gedi, earthdata, raster, icesat2

In [ ]:
%%time
sliderule.init("localhost", organization=None, verbose=True)

In [ ]:
poly_geojson = '/data/3dep/wrzesien_mountain_snow_sieve_poly.geojson'
bbox_gdf = gpd.read_file(poly_geojson)
bbox_gdf.plot()

In [ ]:
bbox_gdf_simple = bbox_gdf.copy()
bbox_gdf_simple['geometry'] = bbox_gdf_simple.simplify(tolerance=0.01)
bbox_gdf_simple = bbox_gdf_simple.dissolve()

In [ ]:
region = sliderule.toregion(bbox_gdf_simple, cellsize=0.01)
region['gdf'].plot()

In [ ]:
year = 2022
t0 = f'{year}-10-01T00:00:00Z'
t1 = f'{year+1}-09-30T23:59:59Z'
time_start = f'{year}-10-01'
time_end = f'{year+1}-09-30'

In [ ]:
%%time
catalog_file = '/data/3dep/wrzesien_mountain_snow_sieve_catalog.geojson'

earthdata.set_max_resources(999999)

if not os.path.exists(catalog_file):
    # usgs3dep_catalog = earthdata.tnm(short_name='Digital Elevation Model (DEM) 1 meter', time_start=time_start, time_end=time_end, polygon=region['poly'])
    # usgs3dep_catalog = earthdata.tnm(short_name='Digital Elevation Model (DEM) 1 meter', time_start=t0, time_end=t1, polygon=region['poly'])
    usgs3dep_catalog = earthdata.tnm(short_name='Digital Elevation Model (DEM) 1 meter', polygon=region['poly'])

    # Save usgs3dep_catalog to a file
    with open(catalog_file, 'w') as file:
        file.write(usgs3dep_catalog)
else:
    with open(catalog_file, 'r') as file:
        usgs3dep_catalog = file.read()

In [ ]:
import json
d = json.loads(usgs3dep_catalog)
len(d['features'])

In [ ]:
d['features'][0]

In [ ]:
parquet_file = '/data/3dep/wrzesien_snow.parquet'
poi_file     = '/data/3dep/wrzesien_snow_poi.txt'

is2_parms = {
    "asset": "icesat2",
    "poly": region['poly'],
    "raster": region['raster'],
    "max_resources": 999999,
    "timeout": 6000,
    "srt": 0,
    "len": 40,
    "res": 20,
    "cnf": 4,
    "atl08_class": ["atl08_ground"],
    "maxi": 6,
    "ats": 20.0,
    "cnt": 10,
    "H_min_win": 3.0,
    "sigma_r_max": 5.0,
    "output": {"path": "/data/3dep/wrzesien_snow.parquet", "format": "parquet", "as_geo": True, "open_on_complete": True},
    # "output": {"path": parquet_file, "format": "parquet", "as_geo": True, "open_on_complete": True},
    # "samples": {"3dep": {"asset": "usgs3dep-1meter-dem", "use_poi_time": True, "catalog": usgs3dep_catalog, "single_stop": True, "t0": t0, "t1": t1}},
    # "samples": {"3dep": {"asset": "usgs3dep-1meter-dem", "use_poi_time": True, "catalog": usgs3dep_catalog, "single_stop": False, "t0": t0, "t1": t1}},
    "t0":t0, "t1":t1   #This is for temporal filtering of ATL08 granules
}

In [ ]:
%%time
resources = earthdata.search(is2_parms)

In [ ]:
%%time
is2_ground = icesat2.atl06p(is2_parms, height_key='h_mean', resources=resources[:1])

In [ ]:
# generate_poi = False
generate_poi = True

if generate_poi and os.path.exists(parquet_file):
    gdf = gpd.read_parquet(parquet_file)
    print(gdf.columns)

    # Ensure the geometry column is in the correct format (Point)
    if not gdf.geometry.geom_type.isin(['Point']).all():
        raise ValueError("All geometries must be of Point type")

    # Open the output file and write the coordinates
    with open(poi_file, mode='w') as txt_file:
        for index, row in gdf.iterrows():
            # Extract longitude and latitude from the geometry column
            longitude = row.geometry.x
            latitude = row.geometry.y
            txt_file.write(f"{longitude} {latitude}\n")

In [ ]:
#is2_ground

In [ ]:
is2_ground['d'] = is2_ground['3dep.value'].notna()

In [ ]:
is2_ground['d'].value_counts()